# What is Selenium?

## 1

What environment do we use?

selenium docker?

See [What is Docker?](What%20is%20Docker.ipynb)



In [4]:
var importer = require('../Core');
var fs = require('fs');

// add some run commands to the bash script
function bashToRun(code) {
    return code.split('\n').filter(n => n.trim().length > 0).map(l => 'RUN ' + l.trim())
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');
}

// create a selenium Dockerfile with a vnc connection
function seleniumDocker(outputFile) {
    const r = importer.interpret([
        'run selenium inside docker',
        // add some extra services
        'linux dev tools',
        'vnc html',
        'vnc docker'
    ]);
    // convert some results to Docker RUN commands instead of bash
    r[1].code = bashToRun(r[1].code);
    r[2].code = bashToRun(r[2].code);
    // save the Dockerfile
    fs.writeFileSync(outputFile, r.map(r => r.code).join('\n'));
    return r;
};
module.exports = seleniumDocker;


[Function: seleniumDocker]

How to set up selenium server?

In [5]:
var importer = require('../Core');
var path = require('path');
var fs = require('fs');
var execCmd = importer.import('spawn child process');

var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));

function getSeleniumServer(name = 'act-selenium') {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') {
            throw err;
        }
    }
    seleniumDocker(DOCKERFILE);
    return execCmd('docker ps -a')
        .then(r => {
            if (r[0].indexOf(name) > -1) {
                return execCmd('docker stop ' + name)
                    .then(r => new Promise(resolve =>
                        setTimeout(() => resolve(r), 1000)))
                    .then(() => execCmd('docker rm ' + name));
            }
        })
        .then(() => new Promise(resolve =>
            setTimeout(() => resolve(), 1000)))
        .then(() => {
            var build = 'docker build -t ' + name + ' "'
                + path.dirname(DOCKERFILE) + '"\n'
                + 'docker run --shm-size=3g -d '
                + '--name ' + name + ' '
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads '
                + name + '\n';
            return execCmd(build)
        })
        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)))
        .catch(e => console.log(e))
};
module.exports = getSeleniumServer;
getSeleniumServer;



[Function: getSeleniumServer]

In [6]:
var importer = require('../Core');
var {getSeleniumServer} = importer.import([
    'What is Selenium.ipynb[what is selenium]',
    'set up selenium server'
]);

$$.async();
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



CONTAINER ID        IMAGE                         COMMAND                  CREATED
             STATUS                      PORTS                                                                                                                                            NAMES
836cc1922516        b18366ef977d                  "/opt/bin/entry_poin…"   2 days ago          Exited (255) 24 hours ago   0.0.0.0:3000->3000/tcp, 0.0.0.0:4200->4200/tcp, 0.0.0.0:4444->4444/tcp, 0.0.0.0:5900->5900/tcp, 0.0.0.0:6080->6080/tcp, 0.0.0.0:8888->8888/tcp   act-selenium
6772e0f45ce4        0278f28bc63f                  "/bin/sh -c 'RUN mkd…"   2 days ago          Exited (127) 2 days ago                                                                                                                                                      competent_clarke
df878c878e11        0278f28bc63f                  "/bin/sh -c 'mkdir /…"   2 weeks ago         Exited (1) 2 weeks ago                                          

[ 'Sending build context to Docker daemon    512kB\r\r\nStep 1/25 : FROM selenium/standalone-chrome-debug\n ---> 593e47f379db\nStep 2/25 : EXPOSE 4200\n ---> Using cache\n ---> 39b5a98f4dc5\nStep 3/25 : EXPOSE 4444\n ---> Using cache\n ---> 3ee9ec07c44f\nStep 4/25 : EXPOSE 3000\n ---> Using cache\n ---> 1d57d4521ce3\nStep 5/25 : ENV DBUS_SESSION_BUS_ADDRESS /dev/null\n ---> Using cache\n ---> 7249cd85d0a3\nStep 6/25 : ENV CHROME_USER_DATA_DIR /usr/profile\n ---> Using cache\n ---> 61abfc7a8627\nStep 7/25 : WORKDIR /home/seluser\n ---> Using cache\n ---> 70e89c34910a\nStep 8/25 : USER root\n ---> Using cache\n ---> 30c904df1014\nStep 9/25 : RUN mkdir /usr/profile\n ---> Using cache\n ---> 9b0c20964a09\nStep 10/25 : RUN mkdir /usr/downloads\n ---> Using cache\n ---> 97afce9e8757\nStep 11/25 : RUN chown seluser:seluser -R /usr/profile\n ---> Using cache\n ---> b1ff20b3a84f\nStep 12/25 : RUN chown seluser:seluser -R /usr/downloads\n ---> Using cache\n ---> 89ed8bb6fac8\nStep 13/25 : RUN ch

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [2]:
function urlEncode(obj) => {
    return Object.keys(obj).map(k => encodeURIComponent(k) + '=' + encodeURIComponent(obj[k])).join('&');
}

var vncIframe = (options = {
    password: 'secret',
    host: 'localhost',
    port: 6080,
    autoconnect: true,
    resize: 'downscale',
    view_only: true,
    reconnect: true
}) => `
<div style="display:block; width:100%; padding-bottom:69%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="//local:6080/vnc.html?${urlEncode(options)}"></iframe></div>
`;
module.exports = vncIframe;
vncIframe;



[Function: vncIframe]

In [3]:
$$.mime({'text/html': vncIframe()});


## 3

How to run a selenium cell on the Docker machine?



In [1]:
var importer = require('../Core');
var createWebdriverClient = importer.import('webdriver client');

function runSeleniumCell(search, newWindow = true) {
    var client = createWebdriverClient();
    var ctx = {client};
    var onlyOneWindow;
    var {
        getCredentials,
        getAllXPath,
        getAllUntil,
        resizeWindow,
        onlyOneWindow,
        getAllSessionUrls,
        clickSpa,
    } = importer.import([
        'decrypt password',
        'all elements xpath',
        'get all elements until',
        'resize selenium window',
        'only one window',
        'get all session urls',
        'click spa link',
    ], ctx);
    var formUtilities = importer.import('form utilities', ctx);
    return client
        .getActiveSessions()
        .then(() => (newWindow
                     ? client
                     .then(() => onlyOneWindow())
                     .then(() => getAllSessionUrls())
                     : client))
        .then(() => {
            return importer.import(search, Object.assign(ctx, {
                client: client,
                browser: client,
                getCredentials,
                getAllXPath,
                getAllUntil,
                resizeWindow,
                clickSpa,
            }, formUtilities))
        })
};
module.exports = runSeleniumCell;


[Function: runSeleniumCell]

In [2]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async()
runSeleniumCell('test docker selenium')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost
[ 1511645427983, 'bd9165cacc918c80b58e1573bbac7a92' ]
[14:31:03]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92/window_handle"
[14:31:03]  COMMAND	POST 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92/window"
[14:31:03]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92"
[ 1511645428050, '3133fe0feb95113004265446032ad1cb' ]
[14:31:03]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window_handle"
[14:31:03]  COMMAND	POST 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window"
[14:31:03]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb"
[ 'bd9165cacc918c80b58e1573bbac7a92',
  '3133fe0feb95113004265446032ad1cb' ]
[14:31:03]  COMMAND	GET 	 "/wd/hub/session/bd9165cacc918c80b58e1573bbac7a92"
bd9165cacc918c80b58e1573bbac7a92
[14:31:03]  COMMAND	GET 	 "/wd/hub/session/3133fe0feb95113004265446032ad1cb/window_handle"
[14:31:03]  COMMAND	POST 	 "/wd/hub/session/3133fe0feb95113004265446032a

{ sessionId: 'bd9165cacc918c80b58e1573bbac7a92',
  status: 0,
  value: null }

Test docker selenium?

In [11]:
function testLive() {
    return client
        .url('https://purchasesprint.actops.com')
        .click('[href*="/auth"], [routerlink*="/auth"]');
}
module.exports = testLive();



ReferenceError: client is not defined


TODO:

Auto fill fields in a live browser using the 

Transfer login state to current browser?

